# Core concepts

## Predefined atlases, parcellations, and reference spaces

The `siibra.core` module provides classes for the core concepts of brain atlases. These include 

 - the `Atlas` class as the basic entry point for working with a brain atlas 
 - the `Parcellation` class, giving access to a particular brain segregation scheme
 - the `Space` class, givin access to a particular reference space
 
The above classes are all derived from the basic `AtlasConcept` class, and represent semantic concepts. They are special in the sense that `siibra` automatically builds a registry with predefined objects for each of them as soon as you import the package. To configure the predefined objects, siibra will retrieve configuration details from EBRAINS and some other online resources when you import the package: 

In [ ]:
import siibra

To performing this bootstrap process, an internet connection is required to import siibra the first time. However, the configuration information is then stored in a local cache folder on your system, so after the first time you will be able to import your `siibra` package without being online.

## Accessing predefined atlases

Siibra represents the core concepts via the registries  `siibra.atlases`, `siibra.parcellations`, and `siibra.spaces`. Elements in each registry can be accessed in different ways:

 - You can iterate over all objects
 - An integer index gives sequential access to individual elements
 - A string index will be matched against the name or key of objects. If it does not match exactly, an inexact string matching will be used to see if a unique entry can be found.
 - Object keys can be tab-completed as attributes of the registry

Let's try this out for the `siibra.atlases` registry.

In [ ]:
# Which atlases are provided? We can iterate over objects in the registry.
for atlas in siibra.atlases:
    print(repr(atlas),"-",atlas.name)

In [ ]:
# Access the first element in the registry
siibra.atlases[0]

In [ ]:
# Access elements by their name as attributes
# with autocompletion by most Python interpreters
siibra.atlases.MULTILEVEL_HUMAN_ATLAS

Note how both alternatives provide the same object. The easiest and recommended way to access items from a registry however, is to use keywords for accessing elements. `siibra` will try to figure out the matching item, or inform you if you need to be more precise.

In [ ]:
# The easiest way: Using string matching of keywords
siibra.atlases['human']

## Accessing parcellations and regions

While the registry `siibra.parcellations` gives access to all available parcellations, the recommended way is to access parcellations via an atlas object. Each atlas object provices a filtered registry of its supported parcellations.

In [ ]:
atlas = siibra.atlases['human']
list(atlas.parcellations)

We can get access to any parcellation object using `atlas.get_parcellation`, or of course by directly using the registry:

In [ ]:
jubrain = atlas.get_parcellation('julich') # will return the latest version per default

# let's look at some metadata
print("Name:    ",jubrain.name)
print("Id:      ",jubrain.id)
print("Modality:",jubrain.modality)
print()
print(jubrain.description)
print()
for p in jubrain.publications:
    print(p['citation'])


The resulting parcellation is a semantic object. It does not itself represent a map, but rather the defintion of the parcellation. Thus it provides

- the region hierarchy
- functions to find and access regions
- functions to access different forms of maps

Let's search a region.

In [ ]:
# search regions known by the parcellation
for r in jubrain.find_regions('amygdala'):
    print(r.name)

As you see, areas often appear three times: Julich-Brain defines them separately for the left and right hemisphere, and additionally defines a common parent region. The parent object actually represents the corresponding subtree. We can more easily access concrete regions by using `decode_region` instead of `find_regions` - it aims to resolve a single element instead of returning all possible candidates.

In [ ]:
# the whole amygdala subtree
jubrain.decode_region('amygdala')

In [ ]:
# only VTM, a small subtree
jubrain.decode_region('VTM')

In [ ]:
# the exact left VTM - this is not a tree, it is a leaf of the region hierarchy
jubrain.decode_region('vtm left')

To search for regions, we need not explicitly fetch the parcellation object. The atlas object provides a similar function, however, it will return matching regions from all parcellations it knows.

In [ ]:
# search all regions known by the atlas
for r in atlas.find_regions('amygdala'):
    print(f"{r.name:30.30} {r.parcellation}")

## Accessing maps

Different from the above semantic objects - atlases, spaces, parcellations, regions - a parcellation map or region map is a spatial object. In some atlases, parcellations are defined in different spaces, so `siibra` expects you to specify one, or will use a default and inform you. Let's get the maximum probability map of the Julich-Brain in the MNI152 space to see how that works.

In [ ]:
mpm = atlas.get_map(space="mni152",parcellation="julich")
type(mpm)

### Labelled parcellation maps

The returned map still does not contain image data. This is because `siibra` uses a lazy strategy for loading data. To access the actual image data, we call the `fetch()` method. We will use the excellent `nilearn` library for plotting the map. It plots in the MNI152 space by default, so as long as we use this space it works easily.

In [ ]:
from nilearn import plotting
plotting.plot_stat_map(mpm.fetch())

Now this is only the left hemisphere! This is because Julich-Brain ships the left and right hemispheres in different volumes, so corresponding regions can use the same label index and still be distinguished. In fact, if a parcellation provides multiple maps, we can iterate them using `fetchall()`:

In [ ]:
for img in mpm.fetchall():
    plotting.plot_stat_map(img)

### Probabilistic maps

Julich-Brain, like some other parcellations, is a probabilistic map. The labelled volumes above are just a simplified representation, displaying for each voxel the btain region of highes probability. We can access the much richer information of the probability maps, which provide a continuous distribution in the reference space for each parcticular region.

We received the labelled volumes above because `siibra` uses labelled volumes as the default map type. To retrieve probability maps, we explictiely use `siibra.maptype.CONTINUOUS`.

In [ ]:
pmaps = atlas.get_map('mni152','julich',maptype='continuous')

Again, we can iterate over all >300 pmaps using `fetchall()`. For simplicity, we will just a random index here.

In [ ]:
pmap = pmaps.fetch(mapindex=102)
plotting.plot_stat_map(pmap)

### Linking region objects with map and label indices

Now you migh wonder which region this refers to. Of course, the parcellation objects in `siibra` help you to safely translate indices into regions, and vice versa:

In [ ]:
# which region corresponds to map index 10?
pmaps.decode_label(mapindex=102)

In [ ]:
# vice versa, what is the index of that region?
pmaps.decode_region('hoc5 left')

In general, a region can be linked to a map index (the index of the image volume) and to a label index (the color in a labelled parcellation map, which does not apply to continuous maps). So, in order to find the label index of CM left in the maximum probability map, we would do exactly the same but get a different index:

In [ ]:
mpm.decode_region("hoc5 left")

We can verify the index by thresholding the MPM with the label index.

In [ ]:
# fetch the left hemishpere maximum probability map
mpm_l = mpm.fetch()

# build a mask by thresholding the label index reported for hoc5
import numpy as np
from nibabel import Nifti1Image
A = mpm_l.get_fdata()
A[A!=6] = 0
mask = Nifti1Image(A,mpm_l.affine)
plotting.plot_roi(mask)

Of course, in order to get a regional mask, you need not take these steps. `siibra`'s region objects can do that right away.

In [ ]:
mask = atlas.get_region('hoc5 left').build_mask("mni152")
plotting.plot_roi(mask)